In [28]:
import numpy as np
from nltk.util import ngrams

import tensorflow as tf
from keras import backend as K
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import Dropout, Dense, Flatten, BatchNormalization, LSTM, Input, concatenate
from keras.models import Sequential, Model
from keras.optimizers import Nadam

# Data Prep

In [167]:
def get_file_names(mypath):
    from os import listdir
    from os.path import isfile, join
    return [f for f in listdir(mypath) if isfile(join(mypath, f))]


def get_notes(notes_path):
    all_song_notes = []
    file_names = get_file_names(notes_path)
    
    for file_name in file_names:
        file_notes = []
        with open(notes_path + file_name, 'r') as file:
            lines = file.readlines()
            file_notes = np.asarray([x.replace("\n", "").split(" ") for x in lines[3:]]).astype("float32")
            #file_notes = file_notes[: -1] # removing last note since there is not next note
        all_song_notes.append(file_notes)
    return np.asarray(all_song_notes)


notes_by_song = get_notes("timings/")

In [238]:
def get_binary_rep(arrow_values):
    return (((arrow_values.astype(int)[:,None] & (1 << np.arange(4)))) > 0).astype(int)


def create_tokens(timings):
    tokens = np.zeros((timings.shape[0], 3))
    tokens[0][0] = 1 # set start token
    next_note_token = np.append(timings[1:] - timings[:-1], np.asarray([0]))
    prev_note_token = np.append(np.asarray([0]),  next_note_token[: -1])
    tokens[:, 1] = prev_note_token.reshape(1, -1)
    tokens[:, 2] = next_note_token.reshape(1, -1)
    return tokens[:-1].astype("float32")


def get_notes_ngram(binary_steps, lookback):
    padding = np.zeros((look_back, binary_steps.shape[1]))
    data_w_padding = np.append(padding, binary_steps, axis = 0)
    return np.asarray(list(ngrams(data_w_padding, look_back)))[:-1]


def data_prep(notes_by_song, lookback = 5):
    from sklearn.preprocessing import OneHotEncoder
    encoder = OneHotEncoder(categories='auto', sparse = False).fit(np.asarray(range(1, 16)).reshape(-1, 1))
    
    all_arrows = []
    all_tokens = []
    all_labels = []
    
    for notes in notes_by_song:
        binary_notes = get_binary_rep(notes[:, 0][:-1])
        
        notes_ngram = get_notes_ngram(binary_notes, lookback)
        tokens = create_tokens(notes[:, 1])
        labels = encoder.transform(notes[:, 0][:-1].reshape(-1, 1))
        
        all_arrows.append(notes_ngram)
        all_tokens.append(tokens)
        all_labels.append(labels)
        
    return np.concatenate(all_arrows), np.concatenate(all_tokens), np.concatenate(all_labels)

lookback = 256
all_arrows, all_tokens, all_labels = data_prep(notes_by_song, lookback = lookback)
all_arrows.shape, all_tokens.shape, all_labels.shape

((24406, 5, 4), (24406, 3), (24406, 15))

# Training

In [239]:
from sklearn.model_selection import train_test_split

arrows_train, arrows_test, labels_train, labels_test = \
    train_test_split(all_arrows, all_labels, test_size=0.2, random_state = 42, shuffle = True)

tokens_train, tokens_test, _, _ = \
    train_test_split(np.expand_dims(all_tokens, axis = 1), all_labels, random_state = 42, test_size=0.2, shuffle = True)

In [245]:
def build(arrows_shape, token_shape, output_shape, lookback, silent = True):
    arrows = Input(shape = (arrows_shape[1], arrows_shape[2],))
    tokens = Input(shape = (token_shape[1], token_shape[2],))
    
    x = LSTM(lookback, kernel_initializer='glorot_normal', return_sequences = True)(arrows)
    x = LSTM(lookback)(x)
    x = Model(inputs = arrows, outputs = x)
    
    y = LSTM(lookback, kernel_initializer='glorot_normal', return_sequences = True)(tokens)
    y = LSTM(lookback)(y)
    y = Model(inputs = tokens, outputs = y)
    
    combined = concatenate([x.output, y.output])
    
    z = Dense(256, kernel_initializer='glorot_normal', activation = "relu")(combined)
    z = Dropout(0.5)(z)
    z = Dense(output_shape[1], activation = "softmax")(z)
    
    model = Model(inputs = [x.input, y.input], outputs = z)
    model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])
    
    if not silent: 
        model.summary()
    
    return model

In [241]:
model = build(arrows_train.shape, tokens_train.shape, labels_train.shape, lookback = lookback, silent = False)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_27 (InputLayer)           (None, 5, 4)         0                                            
__________________________________________________________________________________________________
input_28 (InputLayer)           (None, 1, 3)         0                                            
__________________________________________________________________________________________________
lstm_44 (LSTM)                  (None, 5, 256)       267264      input_27[0][0]                   
__________________________________________________________________________________________________
lstm_46 (LSTM)                  (None, 1, 256)       266240      input_28[0][0]                   
__________________________________________________________________________________________________
lstm_45 (L

In [244]:
batch_size = lookback
model = build(arrows_train.shape, tokens_train.shape, labels_train.shape, lookback = lookback, silent = True)

callbacks = [EarlyStopping(monitor='val_loss', patience=10, verbose=0),
             ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)]

history = model.fit([arrows_train, tokens_train], 
                    labels_train,
                    validation_data=([arrows_test, tokens_test], labels_test),
                    epochs = 100,
                    callbacks = callbacks,
                    batch_size= batch_size,
                    verbose = 1)

Train on 19524 samples, validate on 4882 samples
Epoch 1/100
19524/19524 [==============================] - 13s 653us/step - loss: 1.8301 - acc: 0.2567 - val_loss: 1.6300 - val_acc: 0.3058
Epoch 2/100
19524/19524 [==============================] - 6s 310us/step - loss: 1.4527 - acc: 0.4452 - val_loss: 1.2988 - val_acc: 0.5248
Epoch 3/100
19524/19524 [==============================] - 6s 326us/step - loss: 1.3186 - acc: 0.5221 - val_loss: 1.2682 - val_acc: 0.5459
Epoch 4/100
19524/19524 [==============================] - 6s 321us/step - loss: 1.2725 - acc: 0.5378 - val_loss: 1.2113 - val_acc: 0.5623
Epoch 5/100
19524/19524 [==============================] - 6s 314us/step - loss: 1.2466 - acc: 0.5438 - val_loss: 1.1986 - val_acc: 0.5621
Epoch 6/100
19524/19524 [==============================] - 6s 313us/step - loss: 1.2217 - acc: 0.5512 - val_loss: 1.1965 - val_acc: 0.5643
Epoch 7/100
19524/19524 [==============================] - 6s 310us/step - loss: 1.2026 - acc: 0.5603 - val_loss: 1.

# Predicting